In [1]:
from util.mesh import MeshProcessor
from tqdm import tqdm
import numpy as np
import dask
from dask.distributed import Client, progress
import pandas as pd

cell="jrc_hela-2"

client = Client(threads_per_worker=2, n_workers=1)
client.cluster.scale(8)
mp = MeshProcessor(path = "https://janelia-cosem-datasets.s3.amazonaws.com/jrc_hela-2/neuroglancer/mesh/mito_seg",lod = 2, min_branch_length = 100, use_skeletons=True)
lazy_results = []
for i in range(1,422,1):#range(1,87077,100): #
    lazy_results.append(mp.process_mesh(id=i))
results = dask.compute(*lazy_results)
df = pd.DataFrame.from_records(results)
temp = df[["pic_0","pic_1","pic_2"]].to_numpy()>=0
print(temp.all())


/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Programming/mesh_playground/dask-worker-space/worker-dvt3u3r8', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Programming/mesh_playground/dask-worker-space/worker-mc4nlg8v', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Programming/mesh_playground/dask-worker-space/worker-pnml1au8', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Programming/mesh_playground/dask-worker-space/worker-j37_ddj6', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Progr

True


In [4]:
from __future__ import print_function

import argparse
import webbrowser

import neuroglancer
import neuroglancer.cli
import copy
import requests

from IPython.display import IFrame
import numpy as np

#ap = argparse.ArgumentParser()
#neuroglancer.cli.add_server_arguments(ap)
#args = ap.parse_args()
#neuroglancer.cli.handle_server_arguments(args)


viewer = neuroglancer.Viewer()
response = requests.get(f"https://janelia-cosem-datasets.s3.amazonaws.com/{cell}/neuroglancer/mesh/mito_seg/segment_properties/info")
info_file = response.json()
segment_ids = info_file["inline"]["ids"]
mesh_id_to_index_dict = {}
with viewer.txn() as s:
    s.layers["mesh"] = neuroglancer.SegmentationLayer(  # Single MEsh Layer?
        source=f"precomputed://s3://janelia-cosem-datasets/{cell}/neuroglancer/mesh/mito_seg",
    )
    for index,segment_id in enumerate(segment_ids):
        mesh_id_to_index_dict[int(segment_id)] = index
        s.layers["mesh"].segments.add(segment_id)
    s.layout = '3d'

def my_action(s, class_key, color):

    if s.selected_values["mesh"].value:
        selected_mesh_ids = s.selected_values["mesh"].value
        if s.selected_values["mesh"].value:
            new_state = copy.deepcopy(viewer.state)
            new_state.layers['mesh'].segments.remove(selected_mesh_ids)
            class_layer = f'class {class_key}'
            if class_layer not in new_state.layers: #should do it with name since this is a whole object
                new_state.layers[class_layer] = neuroglancer.SegmentationLayer(  # Single MEsh Layer?
                    source=f"precomputed://s3://janelia-cosem-datasets/{cell}/neuroglancer/mesh/mito_seg")
            new_state.layers[class_layer].segments.add(selected_mesh_ids)

            segment_colors = {}
            for segment_id in new_state.layers[class_layer].segments:
                segment_colors[segment_id] = color
            new_state.layers[class_layer].segment_colors = segment_colors
            
            viewer.set_state(new_state)

def fit_and_predict(s):
    class_layers = ["class h", "class j", "class k", "class l"]
    mesh_index_to_class_dict = {}
    state = viewer.state
    for layer in state.layers:
        if layer.name != "mesh": #then is a class layer
            for segment_id in layer.segments:
                class_id = class_layers.index(layer.name)
                mesh_index = mesh_id_to_index_dict[segment_id]
                mesh_index_to_class_dict[mesh_index] = class_id
     
def my_action_h(s):
    my_action(s,"h", "red")
def my_action_j(s):
    my_action(s,"j","gray")
def my_action_k(s):
    my_action(s,"k","blue")
def my_action_l(s):
    my_action(s,"l","magenta")

class_keys = ["h","j","k","l"]
viewer.actions.add("my-action-h", my_action_h)
viewer.actions.add("my-action-j", my_action_j)
viewer.actions.add("my-action-k", my_action_k)
viewer.actions.add("my-action-l", my_action_l)
#viewer.actions.add("my-action-p", fit_and_predict)

with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer["keyh"] = "my-action-h"
    s.input_event_bindings.viewer["keyj"] = "my-action-j"
    s.input_event_bindings.viewer["keyk"] = "my-action-k"
    s.input_event_bindings.viewer["keyl"] = "my-action-l"
    s.input_event_bindings.viewer["keyp"] = "my-action-p"


    s.status_messages["hello"] = "Welcome to this example"

url = viewer
display(IFrame(url, width=1200, height=800))

In [5]:
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier

class FitAndPredict():
    def __init__(self, df, metrics):
        self.metrics = df[metrics].to_numpy()
    
    def fit(self, mesh_index_to_class_dict):
        self.classifier = MLPClassifier(alpha=1, max_iter=1000)
        classes = list(mesh_index_to_class_dict.values())

        mesh_indices = list(mesh_index_to_class_dict.keys())
        data = self.metrics[mesh_indices, :]
        self.scaler = preprocessing.StandardScaler().fit(data)
        data_transformed = self.scaler.transform(data)
        self.classifier.fit(data_transformed, classes)
        print(*zip(data_transformed,classes))
    def predict(self):
        test_transformed = self.scaler.transform(self.metrics)
        class_predictions = self.classifier.predict(test_transformed)
        return class_predictions

viewer.actions.remove("my-action-p",fit_and_predict)
def fit_and_predict(s):
    class_layers = ["class h", "class j", "class k", "class l"]
    mesh_index_to_class_dict = {}
    state = viewer.state
    for layer in state.layers:
        if layer.name != "mesh": #then is a class layer
            for segment_id in layer.segments:
                class_id = class_layers.index(layer.name)
                mesh_index = mesh_id_to_index_dict[segment_id]
                mesh_index_to_class_dict[mesh_index] = class_id
    print(mesh_index_to_class_dict)
    
    fp = FitAndPredict(df, ["ob_normalized_0","ob_normalized_1","ob_normalized_2","pic_normalized_0","pic_normalized_1","pic_normalized_2","longest_path","num_fragments"])
    fp.fit(mesh_index_to_class_dict)
    class_predictions = fp.predict()

    new_state = copy.deepcopy(viewer.state)
    mesh_layer = new_state.layers['mesh']

    colors = ["red", "gray", "blue", "magenta"]
    
    segment_colors = {}
    for segment_id in mesh_layer.segments:
        mesh_index = mesh_id_to_index_dict[int(segment_id)]
        class_id = class_predictions[mesh_index]
        segment_colors[segment_id] = colors[class_id]

    mesh_layer.segment_colors = segment_colors
    viewer.set_state(new_state)

viewer.actions.add("my-action-p", fit_and_predict)

{231: 0, 1: 0, 410: 0, 395: 0, 137: 1, 108: 1, 262: 1, 329: 2, 41: 2, 12: 2, 365: 2}
(array([ 1.21406697,  0.67625925, -0.929886  ,  0.93680731, -0.91387473,
       -0.98014245, -0.77408518, -0.47012936]), 0) (array([ 1.01946059,  1.24166457, -1.17686417,  1.26787785, -1.29604117,
       -1.17945554, -0.81825239, -0.47012936]), 0) (array([ 1.06417435,  0.77332675, -0.92120474,  0.9651873 , -0.97183997,
       -0.9346103 , -0.8179363 , -0.47012936]), 0) (array([ 1.015296  ,  1.10709144, -1.0959418 ,  0.84645655, -0.77902062,
       -1.00166711, -0.92369118, -0.81489088]), 0) (array([-0.92980681,  0.2682372 ,  0.25009521, -0.46303424,  0.29625198,
        0.87063345,  2.53665901,  2.97748592]), 1) (array([-1.00100381, -0.68677994,  0.8426339 , -0.91418629,  0.90801581,
        0.91620838,  0.4533879 , -0.12536783]), 1) (array([-0.68236703, -1.08288355,  0.93571241, -0.97247216,  0.99726509,
        0.89672301,  0.90624753,  0.56415523]), 1) (array([-0.95204979, -1.13231647,  1.08303225, 